In [377]:
import json
import math
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
from collections import Counter, defaultdict
%matplotlib inline

In [378]:
with open('input23', 'r') as f:
    lines = [l.strip('\n') for l in f.readlines()]

## Part 1

In [379]:
def print_game(rooms, hallway):
    print('#' * (len(hallway) + 2))
    print('#', end='')
    for piece in hallway:
        if piece is None:
            print('.', end='')
        else:
            print(piece[0], end='')
    print('#')
    print('#' * ((len(hallway) + 2 - (2 * len(rooms)) - 1) // 2), end='')
    for i in range(len(rooms)):
        if rooms[i][1] is not None:
            piece = rooms[i][1][0]
        else:
            piece = '.'
        print(f'#{piece}', end='')
    print('#', end='')
    print('#' * ((len(hallway) + 2 - (2 * len(rooms)) - 1) // 2))
    print(' ' * ((len(hallway) + 2 - (2 * len(rooms)) - 1) // 2), end='')
    for i in range(len(rooms)):
        if rooms[i][0] is not None:
            piece = rooms[i][0][0]
        else:
            piece = '.'
        print(f'#{piece}', end='')
    print('#', end='')
    print(' ' * ((len(hallway) + 2 - (2 * len(rooms)) - 1) // 2))
    print(' ' * ((len(hallway) + 2 - (2 * len(rooms)) - 1) // 2), end='')
    for i in range(len(rooms)):
        print('##', end='')
    print('#', end='')
    print(' ' * ((len(hallway) + 2 - (2 * len(rooms)) - 1) // 2))

In [380]:
def get_moves(piece, rooms, hallway, doors):
    debug = False

    piece_type = piece[0]
    home_room = home_rooms[piece_type]
    move_cost = move_costs[piece_type]

    if piece in hallway:
        loc = 'hallway'
    else:
        loc = 'room'

    moves = []
    if loc == 'room':
        # In a room
        for i, room in enumerate(rooms):
            if piece in room:
                room_number = i
                room_index = room.index(piece)
        if room_index == 0 and rooms[room_number][1] is not None:
            # Blocked in room
            if debug: print('blocked in room')
            return []
        if room_number == home_room:
            # In home room
            if room_index == 0 or (room_index == 1 and rooms[room_number][0][0] == piece_type):
                # And either at the end of the room or with other home pieces
                if debug: print('already in home room')
                return []
        # Move into hallway
        room_door = doors[room_number]
        for i in range(room_door + 1, len(hallway)):
            if hallway[i] is None:
                if i not in doors:
                    if room_index == 0:
                        cost = move_cost * 2
                    else:
                        cost = move_cost
                    cost += (i - room_door) * move_cost
                    moves.append(('hallway', i, cost))
            else:
                break
        for i in range(room_door - 1, -1, -1):
            if hallway[i] is None:
                if i not in doors:
                    if room_index == 0:
                        cost = move_cost * 2
                    else:
                        cost = move_cost
                    cost += (room_door - i) * move_cost
                    moves.append(('hallway', i, cost))
            else:
                break
        return moves
        # Ignoring direct moves to other rooms for simplicity
    else:
        # In hallway
        if rooms[home_room][1] is not None:
            # Home room full already
            if debug: print('home room already full')
            return []
        if rooms[home_room][0] is not None and rooms[home_room][0][0] != piece_type:
            # Home room has a different piece in it
            if debug: print('home room has different piece in it')
            return []
        cur_pos = hallway.index(piece)
        if cur_pos > doors[home_room]:
            if sum(hallway[i] is not None for i in range(doors[home_room], cur_pos)) > 0:
                # Blocked path to home room
                if debug: print('path to home room blocked')
                return []
        else:
            if sum(hallway[i] is not None for i in range(cur_pos, doors[home_room] - 1, -1)) > 0:
                # Blocked path to home room
                if debug: print('path to home room blocked')
                return []
        if cur_pos > doors[home_room]:
            cost = (cur_pos - doors[home_room]) * move_cost
        else:
            cost = (doors[home_room] - cur_pos) * move_cost
        if rooms[home_room][0] is None:
            cost += 2 * move_cost
            moves.append(('room', (home_room, 0), cost))
        else:
            cost += move_cost
            moves.append(('room', (home_room, 1), cost))
        return moves

In [381]:
def do_move(piece, move, rooms, hallway):
    new_rooms = deepcopy(rooms)
    new_hallway = deepcopy(hallway)
    if move[0] == 'room':
        # Move to room
        new_hallway[new_hallway.index(piece)] = None
        new_rooms[move[1][0]][move[1][1]] = piece
    else:
        # Move to hallway
        for i, room in enumerate(rooms):
            if piece in room:
                room_number = i
                room_index = room.index(piece)
        new_rooms[room_number][room_index] = None
        new_hallway[move[1]] = piece
    return new_rooms, new_hallway

In [382]:
def get_all_moves(rooms, hallway, doors):
    moves = []
    for piece in ['A1', 'A2', 'B1', 'B2', 'C1', 'C2', 'D1', 'D2']:
        piece_moves = get_moves(piece, rooms, hallway, doors)
        complete_piece_moves = []
        for move in piece_moves:
            complete_piece_moves.append((piece, *move))
        if len(complete_piece_moves) > 0:
            moves.extend(complete_piece_moves)
    return moves

In [383]:
def game_finished(rooms, hallway, doors):
    if sum(h is not None for h in hallway) == 0:
        wrong_room = False
        for piece, i in home_rooms.items():
            if rooms[i][0][0] != piece or rooms[i][1][0] != piece:
                wrong_room = True
                break
        if not wrong_room:
            # Every piece is home
            return True
    return False

In [384]:
def game_stalled(rooms, hallway, doors):
    return len(get_all_moves(rooms, hallway, doors)) == 0

In [385]:
def explore_moves(rooms, hallway, doors, max_cost):
    if game_finished(rooms, hallway, doors):
        return 0
    min_cost = 1e100
    all_moves = sorted(get_all_moves(rooms, hallway, doors), key=lambda m: m[3])
    for move in all_moves:
        piece, ma, mb, move_cost = move
        if move_cost > max_cost:
            continue
        new_rooms, new_hallway = do_move(piece, (ma, mb), rooms, hallway)
        if game_stalled(new_rooms, new_hallway, doors) and not game_finished(new_rooms, new_hallway, doors):
            continue
        resulting_cost = explore_moves(new_rooms, new_hallway, doors, min_cost)
        if resulting_cost == -1:
            continue
        if move_cost + resulting_cost < min_cost:
            min_cost = move_cost + resulting_cost
    if min_cost == 1e100:
        return -1
    return min_cost

In [386]:
lines = [
    '#############',
    '#...........#',
    '###B#C#B#D###',
    '  #A#D#C#A#',
    '  #########',
]

In [387]:
move_costs = {
    'A': 1,
    'B': 10,
    'C': 100,
    'D': 1000,
}
home_rooms = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
}

In [389]:
rooms = [
    ['A1', 'B1'],
    ['D1', 'C1'],
    ['C2', 'B2'],
    ['A2', 'D2'],
]
hallway = [None] * len(lines[1].strip('#'))
doors = [2, 4, 6, 8]

In [375]:
#############
#...B.......#
###B#C#.#D###
  #A#D#C#A#
  #########
test_rooms = [
    ['A1', 'B1'],
    ['D1', 'C1'],
    ['C2', None],
    ['A2', 'D2'],
]
test_hallway = [None, None, None, 'B2', None, None, None, None, None, None, None]
print_game(test_rooms, test_hallway)

#############
#...B.......#
###B#C#.#D###
  #A#D#C#A#  
  #########  


In [390]:
explore_moves(rooms, hallway, doors, 1e100)

KeyboardInterrupt: 

## Part 2